# Image Feature Loader & DB-Saver

Dieses Notebook lädt Bilder, extrahiert Features und speichert sie in einer SQLite-Datenbank.

In [1]:
import os, gc, time, traceback, sqlite3, cProfile, pstats, random
from pathlib import Path
from datetime import datetime

import numpy as np
from tqdm import tqdm
import psutil

from sklearn.decomposition import IncrementalPCA
import umap

# ===== Deine bestehenden Funktionen (NICHT hier implementieren) =====
from loader import image_generator
from features.hash import calc_hash
from features.color_vec import calc_histogram
from features.embedding_vec import extract_embeddings
from image_load import fast_load

# Optional: Header-only Pixel-Check + Resize
try:
    from PIL import Image
    PIL_AVAILABLE = True
except Exception:
    PIL_AVAILABLE = False


# FIRST RUN TO EXTRACT SMALL IMAGES AND MARK BIG IMAGES 

In [2]:


# ========== CONFIG ==========
DB_PATH = r"C:\BIG_DATA\data\database.db"
PHOTO_FOLDER = r"D:\data\image_data"
EMBEDDING_DIR = r"C:\BIG_DATA\embeddings"
EMBEDDING_PCA_DIR = r"C:\BIG_DATA\embeddings_pca"

# Logging / Outputs
LOG_FILE = "verarbeitung_log.txt"
SKIPPED_LOG = "skipped_and_errors.txt"    # fehlerhafte / nicht verarbeitete Bilder
DEFERRED_CSV = "Large_images.csv"         # deferred wegen Größe/Pixeln

# Sharding
IMAGES_PER_TABLE = 50000
TABLE_PREFIX = "image_features_part_"

# Preprocess/Embedding
TARGET_SIZE = (224, 224)      # (W,H) für PIL
EMBED_INPUT_DTYPE = "float32" # "float32" (0..1) oder "uint8"
MICRO_BATCH = 32              # Embedding-Microbatch
MAX_MP = 8                    # >8 Megapixel => defer (nur Header lesen)
MAX_IMAGE_SIZE_BEFORE_REDUCE = 4 * 1024 * 1024  # 4 MB Datei-Threshold

# Pixel-Budget (statisch ODER dynamisch aktivieren)
INITIAL_BATCH_SIZE = 500
PIXEL_BUDGET = INITIAL_BATCH_SIZE * TARGET_SIZE[0] * TARGET_SIZE[1]  # statisch
AUTO_PIXEL_BUDGET_FRACTION = None  # z.B. 0.30 für 30% des freien RAMs; None = aus

# Dirs
Path(EMBEDDING_DIR).mkdir(parents=True, exist_ok=True)
Path(EMBEDDING_PCA_DIR).mkdir(parents=True, exist_ok=True)


# ========== DB ==========
conn = sqlite3.connect(DB_PATH, isolation_level=None)
cursor = conn.cursor()
cursor.execute("PRAGMA journal_mode=OFF;")
cursor.execute("PRAGMA synchronous=OFF;")
cursor.execute("PRAGMA temp_store=MEMORY;")
cursor.execute("PRAGMA mmap_size=0;")  # vermeidet aufgeblähten "sichtbaren" Speicher


# ========== Logging/Helfer ==========
def log_debug(logfile, msg):
    logfile.write(msg + "\n")

def log_skip_or_error(filename, path, reason):
    with open(SKIPPED_LOG, "a", encoding="utf-8") as f:
        f.write(f"{datetime.now()} | {filename} | {path} | {reason}\n")

def mark_deferred_large(filename, path, reason="LARGE_IMAGE_DEFERRED"):
    with open(DEFERRED_CSV, "a", encoding="utf-8") as f:
        f.write(f"{datetime.now()},{filename},{path},{reason}\n")

def print_resource_usage(stage, logfile):
    process = psutil.Process(os.getpid())
    mem = process.memory_info().rss / (1024 ** 2)
    cpu = process.cpu_percent(interval=0.1)
    log_debug(logfile, f"[RESOURCE] {stage} | RAM: {mem:.2f} MB | CPU: {cpu:.2f}%")

def is_large_by_pixels(path, max_mp=MAX_MP):
    if not PIL_AVAILABLE:
        return False
    try:
        with Image.open(path) as im:
            w, h = im.size
        return (w * h) > max_mp * 1_000_000
    except Exception:
        return False

def should_defer(path):
    try:
        if os.path.getsize(path) > MAX_IMAGE_SIZE_BEFORE_REDUCE:
            return True
    except Exception:
        pass
    return is_large_by_pixels(path)

def compute_pixel_budget_from_ram():
    # Dynamisch: z.B. 30% des freien RAMs in Pixel umrechnen
    bytes_per_val = 4 if EMBED_INPUT_DTYPE == "float32" else 1
    channels = 3
    avail_bytes = psutil.virtual_memory().available
    target_frac = AUTO_PIXEL_BUDGET_FRACTION
    target_bytes = int(avail_bytes * target_frac)
    return target_bytes // (channels * bytes_per_val)

if AUTO_PIXEL_BUDGET_FRACTION:
    PIXEL_BUDGET = compute_pixel_budget_from_ram()

def create_table_if_not_exists(table_name):
    cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {table_name} (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            filename TEXT NOT NULL,
            path TEXT NOT NULL,
            color_hist TEXT,
            embedding_path TEXT,
            image_hash TEXT,
            resolution TEXT,
            file_size INTEGER,
            pca_embedding TEXT,
            umap_x REAL,
            umap_y REAL
        )
    ''')
    cursor.execute(f'CREATE INDEX IF NOT EXISTS idx_{table_name}_filename_path ON {table_name}(filename, path);')

def save_batch_to_db(entries, logfile, table_name):
    if not entries:
        return
    log_debug(logfile, f"[DEBUG] Speichere {len(entries)} Einträge in {table_name}...")
    start = time.time()
    cursor.executemany(f"""
        INSERT INTO {table_name}
        (filename, path, color_hist, embedding_path, image_hash, resolution, file_size,
         pca_embedding, umap_x, umap_y)
        VALUES ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, entries)
    dur = time.time() - start
    log_debug(logfile, f"[DEBUG] DB-Speicherung: {dur:.2f}s")

def preprocess_for_model(img_uint8, target_size=TARGET_SIZE):
    # Resize → PIL erwartet (width,height)
    if PIL_AVAILABLE:
        im = Image.fromarray(img_uint8)
        im = im.resize(target_size, Image.BILINEAR)
        arr = np.asarray(im)
    else:
        th, tw = target_size[1], target_size[0]
        y_idx = (np.linspace(0, img_uint8.shape[0]-1, th)).astype(np.int32)
        x_idx = (np.linspace(0, img_uint8.shape[1]-1, tw)).astype(np.int32)
        arr = img_uint8[np.ix_(y_idx, x_idx)]
    if EMBED_INPUT_DTYPE == "float32":
        return (arr.astype(np.float32) / 255.0)
    else:
        return arr.astype(np.uint8)

def to_embed_dtype(arr):
    if EMBED_INPUT_DTYPE == "float32":
        return arr.astype(np.float32) if arr.dtype != np.float32 else arr
    else:
        if arr.dtype == np.float32:
            return (np.clip(arr, 0.0, 1.0) * 255.0).round().astype(np.uint8)
        return arr.astype(np.uint8)

def extract_embeddings_micro(bimgs, chunk=MICRO_BATCH):
    outs = []
    for i in range(0, len(bimgs), chunk):
        outs.append(extract_embeddings(bimgs[i:i+chunk]))
    return np.vstack(outs)

def _unique_npy_path(base_path):
    base, ext = os.path.splitext(base_path)
    if ext.lower() != ".npy":
        base_path = base + ".npy"
        base, ext = os.path.splitext(base_path)
    out = base_path
    c = 1
    while os.path.exists(out):
        out = f"{base}_{c}.npy"
        c += 1
    return out


# ========== Feature-Vorbereitung ==========
def prepare_image_features(filename, path, logfile):
    """
    Defer (Datei/Pixel groß) -> CSV + skip (len==8 Rückgabe).
    Sonst: fast_load (uint8), Hash auf Original, Preprocess (klein) für Histogramm+Embedding.
    """
    try:
        if should_defer(path):
            log_debug(logfile, f"[INFO] Deferred (gross/pixelreich): {filename}")
            mark_deferred_large(filename, path, "LARGE_BY_FILE_OR_PIXELS")
            return (filename, path, None, None, None, None, None, "LARGE_IMAGE_DEFERRED")

        img = fast_load(path)  # deine Loader-Funktion; Erwartung: uint8 ndarray
        if not isinstance(img, np.ndarray):
            raise TypeError("fast_load() must return a NumPy uint8 array.")
        if img.dtype != np.uint8:
            img = np.clip(img, 0, 255).astype(np.uint8)

        resolution = f"{img.shape[1]}x{img.shape[0]}"
        file_size  = os.path.getsize(path)

        # Hash auf Original (robust gg. dtype)
        img_hash = calc_hash(img)

        # RAM-schonend: nur verkleinertes Bild im Batch halten
        img_small = preprocess_for_model(img)     # (224,224,3)
        embed_input = to_embed_dtype(img_small)   # zu erwartetem dtype
        
        color_hist = calc_histogram(img_small, bins=32)
        # Früh stringifizieren (klein halten)
        hist_str = ",".join(str(round(float(v), 6)) for v in np.ravel(color_hist))

        del img
        return (filename, path, embed_input, hist_str, img_hash, resolution, file_size)

    except Exception as e:
        tb = traceback.format_exc()
        reason = f"{e} | Traceback:\n{tb}"
        log_debug(logfile, f"[ERROR] Fehler bei {filename}: {e}")
        log_skip_or_error(filename, path, reason)
        return (filename, path, None, None, None, None, None, reason)


# ========== Globales Finalisieren über alle Shards ==========
def finalize_all_shards_global(
    logfile,
    sample_size: int = 100_000,
    pca_components: int = 100,
    chunk: int = 2048,
):
    """
    Globales Finalisieren über ALLE Tabellen (einmalig am Ende):
    1) IPCA PASS 1: partial_fit auf allen Embeddings (gestreamt)
    2) IPCA PASS 2: transform, PCA-Vektoren speichern, pca_embedding aktualisieren
       (+ Reservoir-Sampling der PCA-Vektoren für UMAP)
    3) UMAP global fit (auf Sample) + PASS 3: transform aller PCA-Vektoren, umap_x/y updaten
    """
    log_debug(logfile, "[DEBUG] === Globales Finalisieren (IPCA+UMAP) startet ===")

    # 0) Alle Shard-Tabellen finden
    cursor.execute(
        "SELECT name FROM sqlite_master WHERE type='table' AND name LIKE ? ORDER BY name ASC",
        (f"{TABLE_PREFIX}%",),
    )
    tables = [r[0] for r in cursor.fetchall()]
    if not tables:
        log_debug(logfile, "[WARN] Keine Shard-Tabellen gefunden – Abbruch.")
        return

    # Helper
    def load_embeddings(paths):
        mats, keep_paths = [], []
        for p in paths:
            try:
                mats.append(np.load(p))
                keep_paths.append(p)
            except Exception as e:
                log_skip_or_error("UNKNOWN", p, f"PCA-Load-Fehler: {e}")
        if not mats:
            return None, []
        X = np.stack(mats, axis=0).astype(np.float32)
        return X, keep_paths

    # PASS 1: IncrementalPCA partial_fit
    ipca = None
    total_vectors = 0
    for tbl in tables:
        cursor.execute(f"SELECT id, embedding_path FROM {tbl}")
        batch_ids, batch_paths = [], []
        while True:
            row = cursor.fetchone()
            if row is None:
                if batch_paths:
                    X, kept = load_embeddings(batch_paths)
                    if X is not None:
                        if ipca is None:
                            nfeat = X.shape[1]
                            ncomp = min(pca_components, nfeat)
                            ipca = IncrementalPCA(n_components=ncomp)
                            log_debug(logfile, f"[DEBUG] IPCA init mit n_components={ncomp} (feat={nfeat})")
                        ipca.partial_fit(X)
                        total_vectors += X.shape[0]
                        del X
                    batch_ids.clear(); batch_paths.clear()
                    gc.collect()
                break

            _id, epath = row
            if epath:
                batch_ids.append(_id)
                batch_paths.append(epath)
                if len(batch_paths) >= chunk:
                    X, kept = load_embeddings(batch_paths)
                    if X is not None:
                        if ipca is None:
                            nfeat = X.shape[1]
                            ncomp = min(pca_components, nfeat)
                            ipca = IncrementalPCA(n_components=ncomp)
                            log_debug(logfile, f"[DEBUG] IPCA init mit n_components={ncomp} (feat={nfeat})")
                        ipca.partial_fit(X)
                        total_vectors += X.shape[0]
                        del X
                    batch_ids.clear(); batch_paths.clear()
                    gc.collect()

    if ipca is None or total_vectors == 0:
        log_debug(logfile, "[WARN] Keine Embeddings gefunden – IPCA konnte nicht trainiert werden.")
        return

    log_debug(logfile, f"[DEBUG] PASS 1 fertig – partial_fit auf {total_vectors} Vektoren.")
    print_resource_usage("Nach IPCA PASS 1", logfile)

    # PASS 2: transform & PCA speichern + Sample sammeln
    rng = random.Random(42)
    sample_Xp = []
    sample_cnt = 0
    total_pca_saved = 0

    for tbl in tables:
        cursor.execute(f"SELECT id, embedding_path FROM {tbl}")
        batch_ids, batch_paths = [], []
        while True:
            row = cursor.fetchone()
            if row is None:
                if batch_paths:
                    X, kept_paths = load_embeddings(batch_paths)
                    if X is not None:
                        Xp = ipca.transform(X)
                        updates = []
                        for (_id, _path), vec in zip(batch_ids, Xp):
                            pca_path = _unique_npy_path(os.path.join(EMBEDDING_PCA_DIR, f"{_id}_pca.npy"))
                            try:
                                np.save(pca_path, vec.astype(np.float32))
                            except Exception as e:
                                log_skip_or_error("UNKNOWN", f"DB_ID={_id}", f"PCA-Save-Fehler: {e}")
                                pca_path = ""
                            updates.append((pca_path, _id))
                            # Reservoir-Sampling für UMAP
                            sample_cnt += 1
                            if len(sample_Xp) < sample_size:
                                sample_Xp.append(vec.copy())
                            else:
                                j = rng.randrange(sample_cnt)
                                if j < sample_size:
                                    sample_Xp[j] = vec.copy()
                        cursor.executemany(f"UPDATE {tbl} SET pca_embedding=? WHERE id=?", updates)
                        conn.commit()
                        total_pca_saved += len(updates)
                        del X, Xp, updates
                    batch_ids.clear(); batch_paths.clear()
                    gc.collect()
                break

            _id, epath = row
            if epath:
                batch_ids.append((_id, epath))
                batch_paths.append(epath)
                if len(batch_paths) >= chunk:
                    X, kept_paths = load_embeddings(batch_paths)
                    if X is not None:
                        Xp = ipca.transform(X)
                        updates = []
                        for (_id2, _path2), vec in zip(batch_ids, Xp):
                            pca_path = _unique_npy_path(os.path.join(EMBEDDING_PCA_DIR, f"{_id2}_pca.npy"))
                            try:
                                np.save(pca_path, vec.astype(np.float32))
                            except Exception as e:
                                log_skip_or_error("UNKNOWN", f"DB_ID={_id2}", f"PCA-Save-Fehler: {e}")
                                pca_path = ""
                            updates.append((pca_path, _id2))
                            sample_cnt += 1
                            if len(sample_Xp) < sample_size:
                                sample_Xp.append(vec.copy())
                            else:
                                j = rng.randrange(sample_cnt)
                                if j < sample_size:
                                    sample_Xp[j] = vec.copy()
                        cursor.executemany(f"UPDATE {tbl} SET pca_embedding=? WHERE id=?", updates)
                        conn.commit()
                        total_pca_saved += len(updates)
                        del X, Xp, updates
                    batch_ids.clear(); batch_paths.clear()
                    gc.collect()

    log_debug(logfile, f"[DEBUG] PASS 2 fertig – {total_pca_saved} PCA-Vektoren gespeichert.")
    print_resource_usage("Nach IPCA PASS 2", logfile)

    if not sample_Xp:
        log_debug(logfile, "[WARN] Kein PCA-Sample vorhanden – UMAP wird übersprungen.")
        return

    # UMAP fit (auf Sample)
    sample_mat = np.vstack(sample_Xp).astype(np.float32)
    reducer = umap.UMAP(n_components=2, metric="euclidean", random_state=42, low_memory=True)
    log_debug(logfile, f"[DEBUG] UMAP Fit auf Sample mit {sample_mat.shape[0]} Vektoren …")
    reducer.fit(sample_mat)
    del sample_mat, sample_Xp
    gc.collect()
    print_resource_usage("Nach UMAP Fit (Sample)", logfile)

    # PASS 3: UMAP transform aller PCA-Vektoren & DB-Update
    total_umap_updated = 0
    for tbl in tables:
        cursor.execute(f"SELECT id, pca_embedding FROM {tbl}")
        batch_ids, pca_paths = [], []
        while True:
            row = cursor.fetchone()
            if row is None:
                if pca_paths:
                    mats, ids_keep = [], []
                    for _id, ppath in batch_ids:
                        try:
                            mats.append(np.load(ppath).astype(np.float32))
                            ids_keep.append(_id)
                        except Exception as e:
                            log_skip_or_error("UNKNOWN", ppath, f"UMAP-PCA-Load-Fehler: {e}")
                    if mats:
                        Xp = np.stack(mats, axis=0)
                        coords = reducer.transform(Xp)
                        updates = [(float(x), float(y), _id) for _id, (x, y) in zip(ids_keep, coords)]
                        cursor.executemany(f"UPDATE {tbl} SET umap_x=?, umap_y=? WHERE id=?", updates)
                        conn.commit()
                        total_umap_updated += len(updates)
                        del Xp, coords, updates, mats
                    batch_ids.clear(); pca_paths.clear()
                    gc.collect()
                break

            _id, ppath = row
            if ppath:
                batch_ids.append((_id, ppath))
                pca_paths.append(ppath)
                if len(pca_paths) >= chunk:
                    mats, ids_keep = [], []
                    for _id2, ppath2 in batch_ids:
                        try:
                            mats.append(np.load(ppath2).astype(np.float32))
                            ids_keep.append(_id2)
                        except Exception as e:
                            log_skip_or_error("UNKNOWN", ppath2, f"UMAP-PCA-Load-Fehler: {e}")
                    if mats:
                        Xp = np.stack(mats, axis=0)
                        coords = reducer.transform(Xp)
                        updates = [(float(x), float(y), _id2) for _id2, (x, y) in zip(ids_keep, coords)]
                        cursor.executemany(f"UPDATE {tbl} SET umap_x=?, umap_y=? WHERE id=?", updates)
                        conn.commit()
                        total_umap_updated += len(updates)
                        del Xp, coords, updates, mats
                    batch_ids.clear(); pca_paths.clear()
                    gc.collect()

    log_debug(logfile, f"[DEBUG] PASS 3 fertig – {total_umap_updated} UMAP-Koordinaten aktualisiert.")
    print_resource_usage("Nach UMAP PASS 3", logfile)
    log_debug(logfile, "[DEBUG] === Globales Finalisieren (IPCA+UMAP) abgeschlossen ===")


# ========== Hauptpipeline (Streaming + PIXEL_BUDGET) ==========
def main():
    logfile = open(LOG_FILE, "a", encoding="utf-8")
    # Logs frisch beginnen (optional anpassen)
    for f in (SKIPPED_LOG, DEFERRED_CSV):
        if os.path.exists(f):
            os.remove(f)

    print_resource_usage("Start", logfile)
    logfile.write(f"[{datetime.now()}] [DEBUG] Start Hauptfunktion\n")

    # Warm-up
    dummy_img = np.zeros((10, 10, 3), dtype=np.uint8)
    _ = calc_hash(dummy_img)
    _ = calc_histogram(dummy_img)
    log_debug(logfile, "[DEBUG] Warm-up abgeschlossen.")

    # Zählen ohne Materialisierung
    total_images = sum(1 for _ in image_generator(PHOTO_FOLDER))
    log_debug(logfile, f"[DEBUG] {total_images} Bilder gefunden")

    # Initiale Tabelle
    table_id = 1
    current_table_name = f"{TABLE_PREFIX}{table_id}"
    create_table_if_not_exists(current_table_name)

    pbar = tqdm(total=total_images, desc="Verarbeitung", unit="Bild")

    gen = image_generator(PHOTO_FOLDER)
    total_inserted = 0

    # Laufender Batch (nur vorverarbeitete Bilder)
    batch_meta, batch_imgs = [], []
    cur_pixels = 0  # Summe der (H*W) der preprocessed Bilder im RAM

    while True:
        try:
            filename, path = next(gen)
        except StopIteration:
            # Ende: evtl. Rest-Flush
            if batch_imgs:
                kept = flush_by_embeddings_and_insert(batch_meta, batch_imgs, logfile, current_table_name)
                total_inserted += kept
                batch_meta.clear(); batch_imgs.clear(); cur_pixels = 0
                gc.collect()
            break

        # Existenz-Check (index-gestützt) – nach (filename,path)
        cursor.execute(f"SELECT 1 FROM {current_table_name} WHERE filename=? AND path=? LIMIT 1", (filename, path))
        if cursor.fetchone():
            log_debug(logfile, f"[DEBUG] Übersprungen (bereits in DB): {filename}")
            log_skip_or_error(filename, path, "Bereits in DB")
            pbar.update(1)
            continue

        # Features vorbereiten
        result = prepare_image_features(filename, path, logfile)
        if len(result) == 8:
            # deferred oder Fehler
            pbar.update(1)
            continue

        fname, p, emb_input, hist_str, img_hash, resolution, size = result
        batch_meta.append((fname, p, hist_str, img_hash, resolution, size))
        batch_imgs.append(emb_input)
        cur_pixels += emb_input.shape[0] * emb_input.shape[1]
        pbar.update(1)

        # Pixel-Budget erreicht? → Flush
        if cur_pixels >= PIXEL_BUDGET:
            kept = flush_by_embeddings_and_insert(batch_meta, batch_imgs, logfile, current_table_name)
            total_inserted += kept

            # Cleanup für nächsten Mini-Batch
            batch_meta.clear(); batch_imgs.clear(); cur_pixels = 0
            gc.collect()
            try:
                import torch
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                    log_debug(logfile, "[DEBUG] CUDA Cache geleert.")
            except Exception:
                pass
            print_resource_usage("Nach PixelBudget-Flush", logfile)

            # Sharding: neue Tabelle, falls Grenze erreicht
            if total_inserted >= table_id * IMAGES_PER_TABLE:
                table_id += 1
                current_table_name = f"{TABLE_PREFIX}{table_id}"
                create_table_if_not_exists(current_table_name)
                log_debug(logfile, f"[DEBUG] Neue Tabelle angelegt: {current_table_name}")

    # === Globales Finalisieren über ALLE Tabellen ===
    finalize_all_shards_global(
        logfile,
        sample_size=100_000,   # ggf. anpassen (RAM/Tempo)
        pca_components=100,
        chunk=2048
    )

    pbar.close()
    logfile.write(f"[{datetime.now()}] ✓ Verarbeitung abgeschlossen. Insgesamt eingefügt: {total_inserted}\n")
    print_resource_usage("Ende", logfile)
    logfile.close()
    conn.close()


def flush_by_embeddings_and_insert(batch_meta, batch_imgs, logfile, table_name):
    """Embeddings (micro-batched) berechnen, speichern, DB-Insert; Rückgabe: #eingefügt."""
    if not batch_imgs:
        return 0

    # Embeddings
    try:
        embs = extract_embeddings_micro(batch_imgs, chunk=MICRO_BATCH)
    except Exception as e:
        log_debug(logfile, f"[ERROR] Embedding-Batch fehlgeschlagen: {e}")
        embs = []
        for (filename, path, hist_str, img_hash, resolution, size), img_small in zip(batch_meta, batch_imgs):
            try:
                embs.append(extract_embeddings([img_small])[0])
            except Exception as ee:
                tb = traceback.format_exc()
                log_skip_or_error(filename, path, f"Embedding-Fehler (single): {ee}\n{tb}")
        if not embs:
            return 0
        embs = np.stack(embs, axis=0)

    # Speichern + DB-Insert
    entries = []
    kept = 0
    for (filename, path, hist_str, img_hash, resolution, size), emb in zip(batch_meta, embs):
        try:
            emb_path = _unique_npy_path(os.path.join(EMBEDDING_DIR, f"{filename}.npy"))
            np.save(emb_path, emb.astype(np.float32))
            entries.append((filename, path, hist_str, emb_path, img_hash, resolution, size, "", 0.0, 0.0))
            kept += 1
        except Exception as e_save:
            tb = traceback.format_exc()
            log_skip_or_error(filename, path, f"Embedding-Save-Fehler: {e_save}\n{tb}")

    save_batch_to_db(entries, logfile, table_name)
    del embs, entries
    return kept


# ========== ENTRYPOINT ==========
if __name__ == "__main__":
    with cProfile.Profile() as pr:
        main()

    with open("profiling_results.txt", "w", encoding="utf-8") as f:
        stats = pstats.Stats(pr, stream=f)
        stats.sort_stats("cumtime").print_stats()
    stats.dump_stats("profiling_results.prof")

Verarbeitung: 100%|██████████| 371202/371202 [3:58:39<00:00, 60.04Bild/s]   z:\CODING\UNI\BIG_DATA\venv\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
Verarbeitung: 100%|██████████| 371202/371202 [4:29:28<00:00, 22.96Bild/s]


durch die bilder neu iterierne und mehrer hash funktionen berechnen weil nur eins (average hash) nicht informativ genug ist

In [4]:
import os
import gc
import sqlite3
import traceback
import numpy as np
import cv2 as cv
from tqdm import tqdm

# ========= CONFIG =========
DB_PATH = r"C:\BIG_DATA\data\database.db"
TABLE_PREFIX = "image_features_part_"
BATCH_SIZE = 2000               # DB-IDs pro Batch
RESIZE_DHASH = 8                # (8+1)x8 -> 64 Bit
RESIZE_PHASH = 32               # 32x32 -> DCT -> 8x8 -> 64 Bit
LOG_FILE = "hash_backfill_log.txt"

# ========= Logging =========
def log(msg: str) -> None:
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(msg + "\n")

# ========= Utils =========
def _bits_to_hex(bits_bool: np.ndarray) -> str:
    """Bool-Array -> Hex (64 Bit -> 16 Hex-Zeichen)."""
    packed = np.packbits(bits_bool.astype(np.uint8), bitorder="big")
    return packed.tobytes().hex()

def _imread_rgb(path: str) -> np.ndarray:
    """Robustes Laden (auch bei Sonderzeichen in Pfaden)."""
    data = np.fromfile(path, dtype=np.uint8)
    img = cv.imdecode(data, cv.IMREAD_COLOR)
    if img is None:
        raise ValueError("cv.imdecode returned None")
    return cv.cvtColor(img, cv.COLOR_BGR2RGB)

# ========= Hash-Funktionen =========
def dhash_hex(img_rgb: np.ndarray, size: int = RESIZE_DHASH) -> str:
    """Difference Hash (dHash), 64 Bit -> 16 Hex."""
    g = cv.cvtColor(img_rgb, cv.COLOR_RGB2GRAY)
    g = cv.resize(g, (size + 1, size), interpolation=cv.INTER_AREA)
    diff = g[:, 1:] > g[:, :-1]
    return _bits_to_hex(diff.reshape(-1))

def phash_hex(img_rgb: np.ndarray, full: int = RESIZE_PHASH, keep: int = 8) -> str:
    """Perceptual Hash (pHash via DCT), 64 Bit -> 16 Hex."""
    g = cv.cvtColor(img_rgb, cv.COLOR_RGB2GRAY)
    g = cv.resize(g, (full, full), interpolation=cv.INTER_AREA).astype(np.float32)
    dct = cv.dct(g)
    dct_low = dct[:keep, :keep].copy()
    dct_low[0, 0] = 0.0
    med = np.median(dct_low)
    bits = dct_low >= med
    return _bits_to_hex(bits.reshape(-1))

# ========= DB-Helfer =========
def get_shard_tables(cursor):
    cursor.execute(
        "SELECT name FROM sqlite_master WHERE type='table' "
        "AND name LIKE ? ORDER BY name ASC",
        (f"{TABLE_PREFIX}%",),
    )
    return [r[0] for r in cursor.fetchall()]

def ensure_columns(conn: sqlite3.Connection, table: str) -> None:
    cur = conn.cursor()
    cur.execute(f"PRAGMA table_info({table});")
    cols = {row[1] for row in cur.fetchall()}
    wanted = {
        "dhash": "TEXT",
        "phash": "TEXT",
    }
    for col, ctype in wanted.items():
        if col not in cols:
            cur.execute(f"ALTER TABLE {table} ADD COLUMN {col} {ctype};")
            log(f"[INFO] Added column {col} to {table}")
    conn.commit()

def count_pending(cur: sqlite3.Cursor, table: str) -> int:
    """Wie viele Zeilen fehlen (mind. einer der Hashes leer)?"""
    cur.execute(
        f"""
        SELECT COUNT(*) FROM {table}
        WHERE (dhash IS NULL OR dhash = '')
           OR (phash IS NULL OR phash = '')
        """
    )
    return int(cur.fetchone()[0])

def fetch_ids_and_paths_needing_hashes(cur: sqlite3.Cursor, table: str, limit: int):
    cur.execute(
        f"""
        SELECT id, path FROM {table}
        WHERE (dhash IS NULL OR dhash = '')
           OR (phash IS NULL OR phash = '')
        LIMIT ?
        """,
        (limit,),
    )
    return cur.fetchall()

def update_hashes(conn: sqlite3.Connection, table: str, rows) -> int:
    """Berechnet Hashes für rows und schreibt sie. Rückgabe: #erfolgreich aktualisiert."""
    cur = conn.cursor()
    updates = []
    for _id, path in rows:
        try:
            img = _imread_rgb(path)
            d = dhash_hex(img)
            p = phash_hex(img)
            updates.append((d, p, _id))
        except Exception as e:
            log(f"[ERROR] {table} id={_id} path='{path}': {e}\n{traceback.format_exc()}")
    if updates:
        cur.executemany(
            f"UPDATE {table} SET dhash=?, phash=? WHERE id=?;",
            updates,
        )
        conn.commit()
    return len(updates)

# (Optional) Indizes
def ensure_indexes(conn: sqlite3.Connection, table: str) -> None:
    cur = conn.cursor()
    cur.execute(f"CREATE INDEX IF NOT EXISTS idx_{table}_dhash ON {table}(dhash);")
    cur.execute(f"CREATE INDEX IF NOT EXISTS idx_{table}_phash ON {table}(phash);")
    conn.commit()

# ========= Main =========
def backfill_all_tables() -> None:
    conn = sqlite3.connect(DB_PATH, isolation_level=None)  # autocommit
    cur = conn.cursor()
    tables = get_shard_tables(cur)
    if not tables:
        log("[WARN] Keine Shard-Tabellen gefunden.")
        conn.close()
        return

    total_updated = 0
    for tbl in tables:
        ensure_columns(conn, tbl)
        pending = count_pending(cur, tbl)

        if pending == 0:
            log(f"[INFO] Nichts zu tun für {tbl}")
            continue

        with tqdm(total=pending, desc=f"{tbl}", unit="img") as pbar:
            # ensure_indexes(conn, tbl)  # optional
            while True:
                rows = fetch_ids_and_paths_needing_hashes(cur, tbl, BATCH_SIZE)
                if not rows:
                    break
                n_ok = update_hashes(conn, tbl, rows)
                total_updated += n_ok
                pbar.update(n_ok)  # Fortschritt = erfolgreich befüllte Zeilen
                gc.collect()

        log(f"[INFO] Fertig für {tbl}")

    log(f"[OK] Backfill abgeschlossen. Insgesamt aktualisiert: {total_updated}")
    conn.close()

if __name__ == "__main__":
    backfill_all_tables()

image_features_part_7: 100%|██████████| 8250/8250 [40:01<00:00,  3.43img/s]


In [6]:
#pca und umap spalten leeren 
import os
import sqlite3
from pathlib import Path
from datetime import datetime

# ======= CONFIG =======
DB_PATH = r"C:\BIG_DATA\data\database.db"
TABLE_PREFIX = "image_features_part_"

# Wo liegen deine PCA-Dateien (zur optionalen Löschung)?
EMBEDDING_PCA_DIR = Path(r"C:\BIG_DATA\embeddings_pca")

# Verhalten
DRY_RUN = False           # True = nur zählen/anzeigen, nichts ändern/löschen
DELETE_PCA_FILES = True  # True = referenzierte PCA-Dateien löschen (nur wenn DRY_RUN=False)

CHUNK = 10000  # Streaminggröße für SELECTs

# ======================

def get_tables(cur, prefix):
    cur.execute("SELECT name FROM sqlite_master WHERE type='table' AND name LIKE ? ORDER BY name ASC",
                (f"{prefix}%",))
    return [r[0] for r in cur.fetchall()]

def exists_safe(p: str) -> bool:
    try:
        return os.path.exists(p)
    except Exception:
        return False

def main():
    conn = sqlite3.connect(DB_PATH, isolation_level=None)
    cur = conn.cursor()
    cur_read = conn.cursor()  # separater Lese-Cursor (sicherer bei Updates)

    tables = get_tables(cur, TABLE_PREFIX)
    if not tables:
        print("Keine Shard-Tabellen gefunden.")
        return

    print(f"[{datetime.now()}] Cleanup startet …")
    print(f"Tabellen: {tables}")
    print(f"DRY_RUN={DRY_RUN}, DELETE_PCA_FILES={DELETE_PCA_FILES}")

    total_rows = total_pca_set = total_umap_set = 0
    total_files_found = total_files_deleted = 0

    for tbl in tables:
        print(f"\n=== {tbl} ===")

        # Basiscounter
        cur.execute(f"SELECT COUNT(*) FROM {tbl}")
        rows_total = cur.fetchone()[0]
        total_rows += rows_total

        cur.execute(f"SELECT COUNT(*) FROM {tbl} WHERE pca_embedding IS NOT NULL AND pca_embedding!=''")
        pca_set = cur.fetchone()[0]
        total_pca_set += pca_set

        cur.execute(f"""SELECT COUNT(*) FROM {tbl}
                        WHERE (umap_x IS NOT NULL AND umap_y IS NOT NULL)
                          AND (umap_x!=0.0 OR umap_y!=0.0)""")
        umap_set = cur.fetchone()[0]
        total_umap_set += umap_set

        print(f"rows_total : {rows_total}")
        print(f"pca_set    : {pca_set}")
        print(f"umap_set   : {umap_set}")

        # PCA-Dateien einsammeln (optional löschen)
        files_found = files_deleted = 0
        if pca_set > 0 and (DELETE_PCA_FILES or DRY_RUN):
            cur_read.execute(f"SELECT pca_embedding FROM {tbl} WHERE pca_embedding IS NOT NULL AND pca_embedding!=''")
            while True:
                rows = cur_read.fetchmany(CHUNK)
                if not rows:
                    break
                for (ppath,) in rows:
                    if not ppath:
                        continue
                    files_found += 1
                    # Aus Sicherheitsgründen nur löschen, wenn Datei im erwarteten Root liegt
                    if (not DRY_RUN) and DELETE_PCA_FILES:
                        try:
                            p = Path(ppath)
                            if p.is_file():
                                # Safety: innerhalb EMBEDDING_PCA_DIR?
                                try:
                                    p.relative_to(EMBEDDING_PCA_DIR)
                                    os.remove(p)
                                    files_deleted += 1
                                except ValueError:
                                    # liegt außerhalb; zur Sicherheit überspringen
                                    print(f"⚠️  Übersprungen (außerhalb Root): {ppath}")
                        except Exception as e:
                            print(f"⚠️  Löschen fehlgeschlagen: {ppath} | {e}")

        total_files_found += files_found
        total_files_deleted += files_deleted
        print(f"PCA-Dateien gefunden : {files_found}")
        if not DRY_RUN and DELETE_PCA_FILES:
            print(f"PCA-Dateien gelöscht : {files_deleted}")

        # DB-Felder leeren
        if DRY_RUN:
            print("DRY_RUN: DB wird NICHT geändert.")
        else:
            # NULL setzen ist sauberer als 0.0/'' (klarer „kein Wert“)
            cur.execute(f"UPDATE {tbl} SET pca_embedding=NULL WHERE pca_embedding IS NOT NULL AND pca_embedding!=''")
            cur.execute(f"UPDATE {tbl} SET umap_x=NULL, umap_y=NULL WHERE (umap_x IS NOT NULL) OR (umap_y IS NOT NULL)")
            conn.commit()
            print("DB-Felder geleert: pca_embedding=NULL, umap_x=NULL, umap_y=NULL")

    print("\n===== Zusammenfassung =====")
    print(f"Tabellen insgesamt      : {len(tables)}")
    print(f"Zeilen gesamt           : {total_rows}")
    print(f"PCA gesetzt (DB) gesamt : {total_pca_set}")
    print(f"UMAP gesetzt (DB) gesamt: {total_umap_set}")
    print(f"PCA-Dateien gefunden    : {total_files_found}")
    if not DRY_RUN and DELETE_PCA_FILES:
        print(f"PCA-Dateien gelöscht    : {total_files_deleted}")

    print(f"\n[{datetime.now()}] Cleanup fertig.")
    conn.close()

if __name__ == "__main__":
    main()


[2025-08-12 22:10:43.133602] Cleanup startet …
Tabellen: ['image_features_part_1', 'image_features_part_2', 'image_features_part_3', 'image_features_part_4', 'image_features_part_5', 'image_features_part_6', 'image_features_part_7']
DRY_RUN=False, DELETE_PCA_FILES=True

=== image_features_part_1 ===
rows_total : 100000
pca_set    : 0
umap_set   : 0
PCA-Dateien gefunden : 0
PCA-Dateien gelöscht : 0


OperationalError: database is locked

In [3]:
import pstats

stats = pstats.Stats("profiling_results.prof")
stats.strip_dirs().sort_stats("cumulative").print_stats(30)     

Sun Aug 10 21:49:22 2025    profiling_results.prof

         351036394 function calls (344465332 primitive calls) in 16175.470 seconds

   Ordered by: cumulative time
   List reduced from 5633 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    4.103    4.103 16175.799 16175.799 621606085.py:442(main)
   321202  103.743    0.000 10380.608    0.032 621606085.py:166(prepare_image_features)
   321202    2.113    0.000 5892.825    0.018 621606085.py:70(should_defer)
  1017866 3975.147    0.004 3975.309    0.004 {built-in method io.open}
   930620 3649.127    0.004 3649.127    0.004 {built-in method nt.stat}
   567406    1.545    0.000 3634.637    0.006 <frozen genericpath>:48(getsize)
      493    2.629    0.005 3576.141    7.254 621606085.py:548(flush_by_embeddings_and_insert)
      493    0.060    0.000 3397.526    6.892 621606085.py:146(extract_embeddings_micro)
     7879   13.108    0.002 3397.328    0.431 embedding_vec.py

Vergleichen wir zunächst Anzahl bearbietete bilder mit Anzahl bilder 

In [2]:
from pathlib import Path

photo_dir = Path(r"C:\BIG_DATA\embeddings")
n_files = sum(1 for p in photo_dir.iterdir() if p.is_file())
print(f"Dateien im Ordner: {n_files}")

Dateien im Ordner: 359474


In [6]:
import sqlite3

DB_PATH = r"C:\BIG_DATA\data\database.db"

# 1) Verbindung öffnen
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# 2) Alle relevanten Tabellen ermitteln
cursor.execute("""
    SELECT name
      FROM sqlite_master
     WHERE type='table'
       AND name LIKE 'image_features_part_%'
""")
tables = [row[0] for row in cursor.fetchall()]

# 3) Zeilen in jeder Tabelle zählen und aufsummieren
total_entries = 0
for table in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    count = cursor.fetchone()[0]
    print(f"{table}: {count} Einträge")
    total_entries += count

conn.close()

print(f"\nGesamtanzahl Einträge: {total_entries}")


image_features_part_1: 100000 Einträge
image_features_part_2: 51500 Einträge
image_features_part_3: 50000 Einträge
image_features_part_4: 50000 Einträge
image_features_part_5: 50004 Einträge
image_features_part_6: 50000 Einträge
image_features_part_7: 8250 Einträge

Gesamtanzahl Einträge: 359754


In [ ]:
import os

PHOTO_FOLDER = r"D:\data\image_data"
IMAGE_EXTS = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff'}  

count = 0
for root, dirs, files in os.walk(PHOTO_FOLDER):
    for fname in files:
        ext = os.path.splitext(fname)[1].lower()
        if ext in IMAGE_EXTS:
            count += 1

print(f"Gefundene Bilder: {count}")


Gefundene Bilder: 371202


überprüfen wie viele bilder erfolgereich PCA embeddings und UMAP koordinaten bekommen haben

In [3]:

import os
import sqlite3
from pathlib import Path
from datetime import datetime

# ========== CONFIG ==========
DB_PATH = r"C:\BIG_DATA\data\database.db"
TABLE_PREFIX = "image_features_part_"
EMBEDDING_DIR = Path(r"C:\BIG_DATA\embeddings")
EMBEDDING_PCA_DIR = Path(r"C:\BIG_DATA\embeddings_pca")

WRITE_CSV = True
CSV_PATH = Path("report_embeddings_pca_umap.csv")

CHUNK = 10000   # DB-Streaminggröße

# ========== HELPERS ==========
def count_files_in_dir(d: Path) -> int:
    try:
        return sum(1 for p in d.iterdir() if p.is_file())
    except Exception:
        return -1

def exists_safe(p: str) -> bool:
    try:
        return os.path.exists(p)
    except Exception:
        return False

def get_tables(cur, prefix):
    cur.execute("SELECT name FROM sqlite_master WHERE type='table' AND name LIKE ? ORDER BY name ASC",
                (f"{prefix}%",))
    return [r[0] for r in cur.fetchall()]

# ========== MAIN AUDIT ==========
def main():
    conn = sqlite3.connect(DB_PATH, isolation_level=None)
    cur = conn.cursor()

    tables = get_tables(cur, TABLE_PREFIX)
    if not tables:
        print("Keine Shard-Tabellen gefunden.")
        return

    print(f"Gefundene Tabellen: {len(tables)} → {tables}")

    # FS Grob-Check
    emb_fs_count = count_files_in_dir(EMBEDDING_DIR)
    pca_fs_count = count_files_in_dir(EMBEDDING_PCA_DIR)
    print(f"[FS] Dateien im EMBEDDING_DIR:   {emb_fs_count}")
    print(f"[FS] Dateien im EMBEDDING_PCA_DIR: {pca_fs_count}")

    # CSV vorbereiten
    if WRITE_CSV:
        import csv
        csvfile = open(CSV_PATH, "w", newline="", encoding="utf-8")
        writer = csv.writer(csvfile, delimiter=";")
        writer.writerow([
            "table", "rows_total",
            "emb_db_set", "emb_fs_missing",
            "pca_db_set", "pca_fs_missing",
            "pca_db_not_set",
            "umap_set", "umap_missing_for_pca",
            "examples_missing_emb_paths", "examples_missing_pca_paths",
            "examples_pca_set_umap_missing_ids"
        ])
    else:
        writer = None

    # Gesamtsummen
    G_rows = G_emb_set = G_pca_set = G_umap_set = 0
    G_emb_fs_missing = G_pca_fs_missing = 0
    G_pca_not_set = G_umap_missing_for_pca = 0

    for tbl in tables:
        print(f"\n=== Prüfe Tabelle: {tbl} ===")

        # Basiscounter aus DB
        cur.execute(f"SELECT COUNT(*) FROM {tbl}")
        rows_total = cur.fetchone()[0]

        cur.execute(f"SELECT COUNT(*) FROM {tbl} WHERE embedding_path IS NOT NULL AND embedding_path != ''")
        emb_db_set = cur.fetchone()[0]

        cur.execute(f"SELECT COUNT(*) FROM {tbl} WHERE pca_embedding IS NOT NULL AND pca_embedding != ''")
        pca_db_set = cur.fetchone()[0]

        cur.execute(f"""
            SELECT COUNT(*) FROM {tbl}
            WHERE (umap_x IS NOT NULL AND umap_y IS NOT NULL) AND (umap_x != 0.0 OR umap_y != 0.0)
        """)
        umap_set = cur.fetchone()[0]

        # Streaming-Check: fehlende Embedding-Dateien
        emb_fs_missing = 0
        miss_emb_examples = []

        cur.execute(f"SELECT id, embedding_path FROM {tbl} WHERE embedding_path IS NOT NULL AND embedding_path != ''")
        while True:
            rows = cur.fetchmany(CHUNK)
            if not rows: break
            for _id, epath in rows:
                if not exists_safe(epath):
                    emb_fs_missing += 1
                    if len(miss_emb_examples) < 5:
                        miss_emb_examples.append(f"id={_id}|{epath}")

        # Streaming-Check: fehlende PCA-Dateien
        pca_fs_missing = 0
        miss_pca_examples = []

        cur.execute(f"SELECT id, pca_embedding FROM {tbl} WHERE pca_embedding IS NOT NULL AND pca_embedding != ''")
        while True:
            rows = cur.fetchmany(CHUNK)
            if not rows: break
            for _id, ppath in rows:
                if not exists_safe(ppath):
                    pca_fs_missing += 1
                    if len(miss_pca_examples) < 5:
                        miss_pca_examples.append(f"id={_id}|{ppath}")

        # PCA gesetzt? nein:
        pca_db_not_set = rows_total - pca_db_set

        # Fälle: PCA gesetzt, aber UMAP fehlt/placeholder
        umap_missing_for_pca = 0
        pca_umap_missing_examples = []
        cur.execute(f"""
            SELECT id, umap_x, umap_y
            FROM {tbl}
            WHERE pca_embedding IS NOT NULL AND pca_embedding != ''
              AND (umap_x IS NULL OR umap_y IS NULL OR (umap_x = 0.0 AND umap_y = 0.0))
        """)
        while True:
            rows = cur.fetchmany(CHUNK)
            if not rows: break
            for _id, x, y in rows:
                umap_missing_for_pca += 1
                if len(pca_umap_missing_examples) < 5:
                    pca_umap_missing_examples.append(f"id={_id}|umap=({x},{y})")

        # Ausgabe pro Tabelle
        print(f"rows_total            : {rows_total}")
        print(f"emb_db_set            : {emb_db_set}")
        print(f"emb_fs_missing        : {emb_fs_missing}")
        print(f"pca_db_set            : {pca_db_set}")
        print(f"pca_db_not_set        : {pca_db_not_set}")
        print(f"pca_fs_missing        : {pca_fs_missing}")
        print(f"umap_set              : {umap_set}")
        print(f"umap_missing_for_pca  : {umap_missing_for_pca}")

        if miss_emb_examples:
            print(f"  Beispiele fehlender EMB-Dateien: {miss_emb_examples}")
        if miss_pca_examples:
            print(f"  Beispiele fehlender PCA-Dateien: {miss_pca_examples}")
        if pca_umap_missing_examples:
            print(f"  Beispiele: PCA gesetzt, UMAP fehlt: {pca_umap_missing_examples}")

        # CSV-Zeile
        if writer:
            writer.writerow([
                tbl, rows_total,
                emb_db_set, emb_fs_missing,
                pca_db_set, pca_fs_missing,
                pca_db_not_set,
                umap_set, umap_missing_for_pca,
                " | ".join(miss_emb_examples),
                " | ".join(miss_pca_examples),
                " | ".join(pca_umap_missing_examples),
            ])

        # Globalsumme
        G_rows += rows_total
        G_emb_set += emb_db_set
        G_pca_set += pca_db_set
        G_umap_set += umap_set
        G_emb_fs_missing += emb_fs_missing
        G_pca_fs_missing += pca_fs_missing
        G_pca_not_set += pca_db_not_set
        G_umap_missing_for_pca += umap_missing_for_pca

    # Gesamt-Zusammenfassung
    print("\n===== GESAMT =====")
    print(f"rows_total            : {G_rows}")
    print(f"emb_db_set            : {G_emb_set}")
    print(f"emb_fs_missing        : {G_emb_fs_missing}")
    print(f"pca_db_set            : {G_pca_set}")
    print(f"pca_db_not_set        : {G_pca_not_set}")
    print(f"pca_fs_missing        : {G_pca_fs_missing}")
    print(f"umap_set              : {G_umap_set}")
    print(f"umap_missing_for_pca  : {G_umap_missing_for_pca}")

    if writer:
        csvfile.close()
        print(f"\nCSV-Report geschrieben nach: {CSV_PATH.resolve()}")

    conn.close()

if __name__ == "__main__":
    print(f"[{datetime.now()}] Audit startet …")
    main()
    print(f"[{datetime.now()}] Audit fertig.")


[2025-08-12 14:58:49.049598] Audit startet …
Gefundene Tabellen: 7 → ['image_features_part_1', 'image_features_part_2', 'image_features_part_3', 'image_features_part_4', 'image_features_part_5', 'image_features_part_6', 'image_features_part_7']
[FS] Dateien im EMBEDDING_DIR:   359474
[FS] Dateien im EMBEDDING_PCA_DIR: 100000

=== Prüfe Tabelle: image_features_part_1 ===
rows_total            : 100000
emb_db_set            : 100000
emb_fs_missing        : 0
pca_db_set            : 0
pca_db_not_set        : 100000
pca_fs_missing        : 0
umap_set              : 0
umap_missing_for_pca  : 0

=== Prüfe Tabelle: image_features_part_2 ===
rows_total            : 51500
emb_db_set            : 51500
emb_fs_missing        : 0
pca_db_set            : 0
pca_db_not_set        : 51500
pca_fs_missing        : 0
umap_set              : 0
umap_missing_for_pca  : 0

=== Prüfe Tabelle: image_features_part_3 ===
rows_total            : 50000
emb_db_set            : 50000
emb_fs_missing        : 0
pca_db

# 2. RUN THROUGH SAVED LARGE FILES  (samller batchsize)


In [1]:

import os, gc, csv, time, sqlite3, traceback, cProfile, pstats
from pathlib import Path
from datetime import datetime

import numpy as np
from tqdm import tqdm
import psutil

# Optional: PIL nur fürs Resize
try:
    from PIL import Image
    PIL_AVAILABLE = True
except Exception:
    PIL_AVAILABLE = False

# ===== Deine bestehenden Funktionen (NICHT hier implementieren) =====
from features.hash import calc_hash
from features.color_vec import calc_histogram
from features.embedding_vec import extract_embeddings
from image_load import fast_load


# ========== CONFIG ==========
DB_PATH = r"C:\BIG_DATA\data\database.db"
EMBEDDING_DIR = r"C:\BIG_DATA\embeddings"
DEFERRED_CSV = r"Z:\CODING\UNI\BIG_DATA\src\Large_images.csv"

LOG_FILE = r"Z:\CODING\UNI\BIG_DATA\src\verarbeitung_log_deferred.txt"
SKIPPED_LOG = r"Z:\CODING\UNI\BIG_DATA\src\skipped_and_errors.txt"

# Sharding
IMAGES_PER_TABLE = 50_000
TABLE_PREFIX = "image_features_part_"

# Preprocess/Embedding
TARGET_SIZE = (224, 224)      # (W,H) für PIL
EMBED_INPUT_DTYPE = "float32" # "float32" (0..1) oder "uint8"
MICRO_BATCH = 32              # Embedding-Microbatch

# Pixel-Budget: max. Summe der (H*W) der KLEINEN Bilder im RAM
BATCH_TARGET = 100            # grobe Ziel-Batchgröße in kleinen Bildern
PIXEL_BUDGET = BATCH_TARGET * TARGET_SIZE[0] * TARGET_SIZE[1]

# Dirs
Path(EMBEDDING_DIR).mkdir(parents=True, exist_ok=True)


# ========== DB ==========
conn = sqlite3.connect(DB_PATH, isolation_level=None)
cursor = conn.cursor()
cursor.execute("PRAGMA journal_mode=OFF;")
cursor.execute("PRAGMA synchronous=OFF;")
cursor.execute("PRAGMA temp_store=MEMORY;")
cursor.execute("PRAGMA mmap_size=0;")

def create_table_if_not_exists(table_name):
    cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {table_name} (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            filename TEXT NOT NULL,
            path TEXT NOT NULL,
            color_hist TEXT,
            embedding_path TEXT,
            image_hash TEXT,
            resolution TEXT,
            file_size INTEGER,
            pca_embedding TEXT,
            umap_x REAL,
            umap_y REAL
        )
    ''')
    cursor.execute(f'CREATE INDEX IF NOT EXISTS idx_{table_name}_filename_path ON {table_name}(filename, path);')

def get_existing_shards():
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name LIKE ? ORDER BY name ASC",
                   (f"{TABLE_PREFIX}%",))
    return [r[0] for r in cursor.fetchall()]

def rows_in_table(table_name):
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    return cursor.fetchone()[0]

def exists_in_any_shard(filename, path, shard_tables):
    for tbl in shard_tables:
        cursor.execute(f"SELECT 1 FROM {tbl} WHERE filename=? AND path=? LIMIT 1", (filename, path))
        if cursor.fetchone():
            return True
    return False

def save_batch_to_db(entries, table_name, logfile):
    if not entries:
        return
    log_debug(logfile, f"[DEBUG] Speichere {len(entries)} Einträge in {table_name} …")
    start = time.time()
    cursor.executemany(f"""
        INSERT INTO {table_name}
        (filename, path, color_hist, embedding_path, image_hash, resolution, file_size,
         pca_embedding, umap_x, umap_y)
        VALUES ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, entries)
    conn.commit()
    log_debug(logfile, f"[DEBUG] DB-Speicherung: {time.time()-start:.2f}s")


# ========== Logging ==========
def log_debug(logfile, msg):
    logfile.write(msg + "\n"); logfile.flush()

def log_skip_or_error(filename, path, reason):
    with open(SKIPPED_LOG, "a", encoding="utf-8") as f:
        f.write(f"{datetime.now()} | {filename} | {path} | {reason}\n")


# ========== Utils ==========
def _unique_npy_path(base_path):
    base, ext = os.path.splitext(base_path)
    if ext.lower() != ".npy":
        base = base + ".npy"
    out = base
    c = 1
    while os.path.exists(out):
        out = f"{base[:-4]}_{c}.npy"
        c += 1
    return out

def preprocess_for_model(img_uint8, target_size=TARGET_SIZE):
    if PIL_AVAILABLE:
        im = Image.fromarray(img_uint8)
        im = im.resize(target_size, Image.BILINEAR)
        arr = np.asarray(im)
    else:
        th, tw = target_size[1], target_size[0]
        y_idx = (np.linspace(0, img_uint8.shape[0]-1, th)).astype(np.int32)
        x_idx = (np.linspace(0, img_uint8.shape[1]-1, tw)).astype(np.int32)
        arr = img_uint8[np.ix_(y_idx, x_idx)]
    if EMBED_INPUT_DTYPE == "float32":
        return (arr.astype(np.float32) / 255.0)
    else:
        return arr.astype(np.uint8)

def to_embed_dtype(arr):
    if EMBED_INPUT_DTYPE == "float32":
        return arr.astype(np.float32) if arr.dtype != np.float32 else arr
    else:
        if arr.dtype == np.float32:
            return (np.clip(arr, 0.0, 1.0) * 255.0).round().astype(np.uint8)
        return arr.astype(np.uint8)

def extract_embeddings_micro(bimgs, chunk=MICRO_BATCH):
    outs = []
    for i in range(0, len(bimgs), chunk):
        outs.append(extract_embeddings(bimgs[i:i+chunk]))
    return np.vstack(outs)


# ========== CSV-Quelle (Large_images.csv) ==========
def iter_deferred_csv(csv_path):
    """
    Erwartetes Format je Zeile: timestamp,filename,path,reason
    (wir ignorieren timestamp/reason und yielden (filename, path))
    """
    if not os.path.exists(csv_path):
        return
    with open(csv_path, "r", encoding="utf-8", newline="") as f:
        r = csv.reader(f)
        for row in r:
            if not row or len(row) < 3:
                continue
            # robust gg. evtl. Header
            if row[0].lower().startswith("timestamp"):
                continue
            ts, filename, path = row[0], row[1], row[2]
            yield filename, path


# ========== Feature-Vorbereitung ==========
def prepare_image_features(filename, path, logfile):
    """
    Lädt großes Bild, verkleinert sofort, berechnet Hash/Histogramm.
    Nur die KLEINE Version wird im Batch gehalten.
    """
    try:
        img = fast_load(path)  # -> uint8 RGB
        if not isinstance(img, np.ndarray):
            raise TypeError("fast_load() must return a NumPy uint8 array.")
        if img.dtype != np.uint8:
            img = np.clip(img, 0, 255).astype(np.uint8)

        resolution = f"{img.shape[1]}x{img.shape[0]}"
        file_size  = os.path.getsize(path)

        img_hash = calc_hash(img)

        img_small = preprocess_for_model(img)
        emb_input = to_embed_dtype(img_small)
        color_hist = calc_histogram(img_small)  # bins=default deiner Funktion
        hist_str = ",".join(str(round(float(v), 6)) for v in np.ravel(color_hist))

        del img
        return (filename, path, emb_input, hist_str, img_hash, resolution, file_size)

    except Exception as e:
        tb = traceback.format_exc()
        reason = f"{e} | Traceback:\n{tb}"
        log_debug(logfile, f"[ERROR] Fehler bei {filename}: {e}")
        log_skip_or_error(filename, path, reason)
        return (filename, path, None, None, None, None, None, reason)


# ========== Flush: Embeddings + Insert ==========
def flush_by_embeddings_and_insert(batch_meta, batch_imgs, logfile, table_name):
    if not batch_imgs:
        return 0
    try:
        embs = extract_embeddings_micro(batch_imgs, chunk=MICRO_BATCH)
    except Exception as e:
        log_debug(logfile, f"[ERROR] Embedding-Chunk fehlgeschlagen: {e}")
        embs = []
        for (filename, path, hist_str, img_hash, resolution, size), img_small in zip(batch_meta, batch_imgs):
            try:
                embs.append(extract_embeddings([img_small])[0])
            except Exception as ee:
                tb = traceback.format_exc()
                log_skip_or_error(filename, path, f"Embedding-Fehler (single): {ee}\n{tb}")
        if not embs:
            return 0
        embs = np.stack(embs, axis=0)

    entries = []
    kept = 0
    for (filename, path, hist_str, img_hash, resolution, size), emb in zip(batch_meta, embs):
        try:
            emb_path = _unique_npy_path(os.path.join(EMBEDDING_DIR, f"{filename}.npy"))
            np.save(emb_path, emb.astype(np.float32))
            entries.append((filename, path, hist_str, emb_path, img_hash, resolution, size, "", 0.0, 0.0))
            kept += 1
        except Exception as e_save:
            tb = traceback.format_exc()
            log_skip_or_error(filename, path, f"Embedding-Save-Fehler: {e_save}\n{tb}")

    save_batch_to_db(entries, table_name, logfile)
    del embs, entries
    return kept


# ========== MAIN ==========
def main():
    logfile = open(LOG_FILE, "a", encoding="utf-8", buffering=1)
    log_debug(logfile, f"[{datetime.now()}] [DEBUG] Start Deferred-Run (kein PCA/UMAP)")
    # Liste existierender Shards ermitteln
    shard_tables = get_existing_shards()
    if not shard_tables:
        shard_tables = [f"{TABLE_PREFIX}1"]
        create_table_if_not_exists(shard_tables[0])

    # In die letzte Tabelle weiter einfügen
    current_table_name = shard_tables[-1]
    current_count = rows_in_table(current_table_name)

    paths_iter = list(iter_deferred_csv(DEFERRED_CSV))
    total_items = len(paths_iter)
    pbar = tqdm(total=total_items, desc="Deferred-Verarbeitung", unit="Bild")

    batch_meta, batch_imgs = [], []
    cur_pixels = 0
    total_inserted = 0

    for filename, path in paths_iter:
        # Dedupe/Existenzcheck über alle Shards
        if exists_in_any_shard(filename, path, shard_tables):
            log_skip_or_error(filename, path, "Bereits in DB (deferred pass)")
            pbar.update(1)
            continue

        result = prepare_image_features(filename, path, logfile)
        if len(result) == 8:
            pbar.update(1)
            continue

        _, p, emb_input, hist_str, img_hash, resolution, size = result
        batch_meta.append((filename, p, hist_str, img_hash, resolution, size))
        batch_imgs.append(emb_input)
        cur_pixels += emb_input.shape[0] * emb_input.shape[1]
        pbar.update(1)

        if cur_pixels >= PIXEL_BUDGET:
            kept = flush_by_embeddings_and_insert(batch_meta, batch_imgs, logfile, current_table_name)
            total_inserted += kept
            current_count += kept

            batch_meta.clear(); batch_imgs.clear(); cur_pixels = 0
            gc.collect()
            try:
                import torch
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                    log_debug(logfile, "[DEBUG] CUDA Cache geleert.")
            except Exception:
                pass

            # Sharding: neue Tabelle nötig?
            if current_count >= IMAGES_PER_TABLE:
                # neue Tabelle anlegen
                next_id = len(shard_tables) + 1
                current_table_name = f"{TABLE_PREFIX}{next_id}"
                create_table_if_not_exists(current_table_name)
                shard_tables.append(current_table_name)
                current_count = 0
                log_debug(logfile, f"[DEBUG] Neue Tabelle angelegt: {current_table_name}")

    # Finaler Flush
    if batch_imgs:
        kept = flush_by_embeddings_and_insert(batch_meta, batch_imgs, logfile, current_table_name)
        total_inserted += kept
        current_count += kept
        batch_meta.clear(); batch_imgs.clear(); cur_pixels = 0
        gc.collect()

    pbar.close()
    log_debug(logfile, f"[{datetime.now()}] ✓ Deferred-Run fertig. Inserts: {total_inserted}")
    logfile.close()
    conn.close()


# ========= ENTRYPOINT =========
if __name__ == "__main__":
    with cProfile.Profile() as pr:
        main()
    with open("profiling_results_deferred.txt", "w", encoding="utf-8") as f:
        stats = pstats.Stats(pr, stream=f)
        stats.sort_stats("cumtime").print_stats()
    stats.dump_stats("profiling_results_deferred.prof")


Deferred-Verarbeitung: 100%|██████████| 62050/62050 [6:08:16<00:00,  2.81Bild/s]    


# FIT AND TRANSFORM PCA AND CALCULATE UMAP KOORDINATES GLOBALLY ON ALL THE IMAGES AT THE SAME TIME

In [7]:
# -*- coding: utf-8 -*-
"""
Globales Finalisieren (IPCA + UMAP) über alle Shards.
Fixes:
- PASS 2: korrektes Mapping (id <-> embedding_path), defekte Pfade werden pro-Item geloggt/übersprungen
- _unique_npy_path: Endung .npy bleibt korrekt erhalten (keine "pfad ohne endung" mehr)
"""

import os, gc, sqlite3, random
from datetime import datetime
from pathlib import Path

import numpy as np
from tqdm import tqdm
import psutil

from sklearn.decomposition import IncrementalPCA
import umap


# ========= CONFIG =========
DB_PATH = r"C:\BIG_DATA\data\database.db"
TABLE_PREFIX = "image_features_part_"
EMBEDDING_PCA_DIR = Path(r"C:\BIG_DATA\embeddings_pca")
LOG_FILE = "verarbeitung_log.txt"
SKIPPED_LOG = "skipped_and_errors.txt"

# Tuning
PCA_COMPONENTS = 64        # 64 oder 100 – kleiner = schneller/leichter
UMAP_SAMPLE_SIZE = 25_000  # Sample fürs UMAP-Fit (Rest wird transformiert)
CHUNK = 4096               # I/O-Chunkgröße (RAM vs. Speed)


# ========= Logging / Helpers =========
def log_debug(f, msg: str):
    f.write(msg + "\n")
    f.flush()

def log_skip_or_error(filename_or_id, path, reason):
    with open(SKIPPED_LOG, "a", encoding="utf-8") as ff:
        ff.write(f"{datetime.now()} | {filename_or_id} | {path} | {reason}\n")

def _unique_npy_path(base_path: str) -> str:
    """
    Gibt einen nicht-belegten .npy-Pfad zurück. Falls base_path bereits .npy hat,
    bleibt die Endung erhalten; sonst wird .npy angehängt.
    """
    base, ext = os.path.splitext(base_path)
    if ext.lower() != ".npy":
        out = base_path + ".npy"
    else:
        out = base_path
    c = 1
    while os.path.exists(out):
        # base ist ohne Endung
        out = f"{base}_{c}.npy"
        c += 1
    return out

def get_tables(cur, prefix):
    cur.execute(
        "SELECT name FROM sqlite_master WHERE type='table' AND name LIKE ? ORDER BY name ASC",
        (f"{prefix}%",),
    )
    return [r[0] for r in cur.fetchall()]

def load_embeddings(paths):
    """Lädt eine Liste von .npy-Pfaden -> (N,D) float32, überspringt defekte. (Nur für PASS 1.)"""
    mats, keep = [], []
    for p in paths:
        try:
            mats.append(np.load(p))
            keep.append(p)
        except Exception as e:
            log_skip_or_error("UNKNOWN", p, f"PCA-Load-Fehler: {e}")
    if not mats:
        return None, []
    X = np.stack(mats, axis=0).astype(np.float32)
    return X, keep


# ========= Finalizer =========
def finalize_all_shards_global(conn, cursor, logfile,
                               sample_size=UMAP_SAMPLE_SIZE,
                               pca_components=PCA_COMPONENTS,
                               chunk=CHUNK):
    log_debug(logfile, "[DEBUG] === Globales Finalisieren (IPCA+UMAP) startet ===")

    # Tabellen finden
    tables = get_tables(cursor, TABLE_PREFIX)
    if not tables:
        log_debug(logfile, "[WARN] Keine Shard-Tabellen gefunden – Abbruch.")
        return
    os.makedirs(EMBEDDING_PCA_DIR, exist_ok=True)

    # eigener Lese-Cursor (SELECT vom UPDATE trennen!)
    cur_read = conn.cursor()

    # -------- totals für Progressbars --------
    total_embed_rows = 0
    for tbl in tables:
        cur_read.execute(f"SELECT COUNT(*) FROM {tbl} WHERE embedding_path IS NOT NULL AND embedding_path != ''")
        total_embed_rows += int(cur_read.fetchone()[0])

    # -------- PASS 1: IPCA partial_fit (global, gestreamt) --------
    ipca = None
    seen = 0
    pbar1 = tqdm(total=total_embed_rows, desc="PASS1 IPCA partial_fit", unit="vec")
    for tbl in tables:
        cur_read.execute(f"SELECT embedding_path FROM {tbl} WHERE embedding_path IS NOT NULL AND embedding_path != ''")
        while True:
            rows = cur_read.fetchmany(chunk)
            if not rows:
                break
            paths = [r[0] for r in rows]
            X, _ = load_embeddings(paths)  # für partial_fit reicht das
            if X is not None:
                if ipca is None:
                    nfeat = X.shape[1]
                    ncomp = min(pca_components, nfeat)
                    ipca = IncrementalPCA(n_components=ncomp)
                    log_debug(logfile, f"[DEBUG] IPCA init mit n_components={ncomp} (feat={nfeat})")
                ipca.partial_fit(X)
                seen += X.shape[0]
                del X
            # Fortschritt nach geplanter Menge im Chunk (auch wenn einzelne fehlschlugen)
            pbar1.update(len(paths))
            gc.collect()
    pbar1.close()

    if ipca is None or seen == 0:
        log_debug(logfile, "[WARN] Keine Embeddings gefunden – IPCA konnte nicht trainiert werden.")
        return

    log_debug(logfile, f"[DEBUG] PASS 1 fertig – partial_fit auf {seen} Vektoren.")
    _print_res("Nach IPCA PASS 1", logfile)

    # -------- PASS 2: transform, PCA speichern, pca_embedding updaten, Sample sammeln --------
    rng = random.Random(42)
    sample_Xp, sample_cnt = [], 0
    saved = 0

    pbar2 = tqdm(total=total_embed_rows, desc="PASS2 IPCA transform+save", unit="vec")
    for tbl in tables:
        cur_read.execute(f"SELECT id, embedding_path FROM {tbl} WHERE embedding_path IS NOT NULL AND embedding_path != ''")
        while True:
            rows = cur_read.fetchmany(chunk)
            if not rows:
                break

            ids   = [r[0] for r in rows]
            paths = [r[1] for r in rows]

            # WICHTIG: pro (id, path) laden -> nur erfolgreiche Paare übernehmen
            mats, keep_ids = [], []
            for _id, p in zip(ids, paths):
                try:
                    mats.append(np.load(p).astype(np.float32))
                    keep_ids.append(_id)
                except Exception as e:
                    log_skip_or_error(f"DB_ID={_id}", p, f"PCA-Load-Fehler: {e}")

            if mats:
                X = np.stack(mats, axis=0)
                Xp = ipca.transform(X)

                updates = []
                for _id_ok, vec in zip(keep_ids, Xp):
                    pca_path = _unique_npy_path(str(EMBEDDING_PCA_DIR / f"{_id_ok}_pca.npy"))
                    try:
                        np.save(pca_path, vec.astype(np.float32))
                    except Exception as e:
                        log_skip_or_error(f"DB_ID={_id_ok}", pca_path, f"PCA-Save-Fehler: {e}")
                        pca_path = ""
                    updates.append((pca_path, _id_ok))

                    # Reservoir-Sampling für UMAP-Fit
                    sample_cnt += 1
                    if len(sample_Xp) < sample_size:
                        sample_Xp.append(vec.copy())
                    else:
                        j = rng.randrange(sample_cnt)
                        if j < sample_size:
                            sample_Xp[j] = vec.copy()

                cursor.executemany(f"UPDATE {tbl} SET pca_embedding=? WHERE id=?", updates)
                conn.commit()
                saved += len(updates)

                del X, Xp, updates, mats
                gc.collect()

            # Fortschritt: geplanter Chunk (auch wenn einzelne fehlschlugen)
            pbar2.update(len(paths))
    pbar2.close()

    log_debug(logfile, f"[DEBUG] PASS 2 fertig – {saved} PCA-Vektoren gespeichert.")
    _print_res("Nach IPCA PASS 2", logfile)

    if not sample_Xp:
        log_debug(logfile, "[WARN] Kein PCA-Sample – UMAP wird übersprungen.")
        return

    # -------- UMAP Fit (auf Sample) --------
    sample_mat = np.vstack(sample_Xp).astype(np.float32)
    reducer = umap.UMAP(n_components=2, metric="euclidean", random_state=42, low_memory=True)
    log_debug(logfile, f"[DEBUG] UMAP Fit auf Sample mit {sample_mat.shape[0]} Vektoren …")
    reducer.fit(sample_mat)
    del sample_mat, sample_Xp
    gc.collect()
    _print_res("Nach UMAP Fit (Sample)", logfile)

    # -------- PASS 3: UMAP transform aller PCA-Vektoren & DB-Update --------
    # wie viele haben jetzt pca_embedding?
    total_pca_rows = 0
    for tbl in tables:
        cur_read.execute(f"SELECT COUNT(*) FROM {tbl} WHERE pca_embedding IS NOT NULL AND pca_embedding != ''")
        total_pca_rows += int(cur_read.fetchone()[0])

    updated = 0
    pbar3 = tqdm(total=total_pca_rows, desc="PASS3 UMAP transform+update", unit="vec")
    for tbl in tables:
        cur_read.execute(f"SELECT id, pca_embedding FROM {tbl} WHERE pca_embedding IS NOT NULL AND pca_embedding != ''")
        while True:
            rows = cur_read.fetchmany(chunk)
            if not rows:
                break

            ids = [r[0] for r in rows]
            ppaths = [r[1] for r in rows]

            mats, keep_ids = [], []
            for _id, p in zip(ids, ppaths):
                try:
                    mats.append(np.load(p).astype(np.float32))
                    keep_ids.append(_id)
                except Exception as e:
                    log_skip_or_error(f"DB_ID={_id}", p, f"UMAP-PCA-Load-Fehler: {e}")

            if mats:
                Xp = np.stack(mats, axis=0)
                coords = reducer.transform(Xp)
                updates = [(float(x), float(y), i) for i, (x, y) in zip(keep_ids, coords)]
                cursor.executemany(f"UPDATE {tbl} SET umap_x=?, umap_y=? WHERE id=?", updates)
                conn.commit()
                updated += len(updates)

                del Xp, coords, updates, mats
                gc.collect()

            pbar3.update(len(rows))
    pbar3.close()

    log_debug(logfile, f"[DEBUG] PASS 3 fertig – {updated} UMAP-Koordinaten aktualisiert.")
    _print_res("Nach UMAP PASS 3", logfile)
    log_debug(logfile, "[DEBUG] === Globales Finalisieren (IPCA+UMAP) abgeschlossen ===")


def _print_res(stage, logfile):
    # kurze Ressourcen-Info
    process = psutil.Process(os.getpid())
    mem = process.memory_info().rss / (1024 ** 2)
    cpu = process.cpu_percent(interval=0.1)
    log_debug(logfile, f"[RESOURCE] {stage} | RAM: {mem:.2f} MB | CPU: {cpu:.2f}%")


# ========= main =========
if __name__ == "__main__":
    os.makedirs(EMBEDDING_PCA_DIR, exist_ok=True)

    conn = sqlite3.connect(DB_PATH, isolation_level=None)
    cursor = conn.cursor()

    # (Optionale) PRAGMAs
    cursor.execute("PRAGMA journal_mode=OFF;")
    cursor.execute("PRAGMA synchronous=OFF;")
    cursor.execute("PRAGMA temp_store=MEMORY;")
    cursor.execute("PRAGMA mmap_size=0;")

    with open(LOG_FILE, "a", encoding="utf-8", buffering=1) as logfile:
        log_debug(logfile, f"[{datetime.now()}] [DEBUG] finalize_only gestartet")
        try:
            finalize_all_shards_global(conn, cursor, logfile,
                                       sample_size=UMAP_SAMPLE_SIZE,
                                       pca_components=PCA_COMPONENTS,
                                       chunk=CHUNK)
        finally:
            log_debug(logfile, f"[{datetime.now()}] [DEBUG] finalize_only beendet")

    conn.close()



PASS2 IPCA transform+save: 100%|██████████| 359754/359754 [34:14<00:00, 175.08vec/s]
z:\CODING\UNI\BIG_DATA\venv\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
PASS3 UMAP transform+update: 100%|██████████| 359754/359754 [27:53<00:00, 214.94vec/s]


In [ ]:
import os, sqlite3
from tqdm import tqdm

DB_PATH = r"C:\BIG_DATA\data\database.db"
TABLE_PREFIX = "image_features_part_"

def get_tables(cur):
    cur.execute("SELECT name FROM sqlite_master WHERE type='table' AND name LIKE ? ORDER BY name ASC",
                (f"{TABLE_PREFIX}%",))
    return [r[0] for r in cur.fetchall()]

def main():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    tables = get_tables(cur)
    for tbl in tables:
        cur.execute(f"SELECT COUNT(*) FROM {tbl} WHERE embedding_path IS NOT NULL AND TRIM(embedding_path)<>''")
        total = int(cur.fetchone()[0])
        if total == 0:
            print(f"{tbl}: 0 embedding_path – übersprungen")
            continue
        ok = missing = 0
        cur.execute(f"SELECT embedding_path FROM {tbl} WHERE embedding_path IS NOT NULL AND TRIM(embedding_path)<>''")
        for (p,) in tqdm(cur.fetchall(), desc=tbl, unit="path", leave=False):
            p = p.strip()
            if os.path.exists(p):
                ok += 1
            elif os.path.exists(p + ".npy"):
                # Falls hier viele Treffer: vorherigen Repair-Job laufen lassen, um in der DB auf '.npy' zu korrigieren
                missing += 1
            else:
                missing += 1
        print(f"{tbl}: embeddings ok={ok}, missing_or_wrong={missing}, total_nonempty={total}")
    conn.close()

if __name__ == "__main__":
    main()


image_features_part_1: embeddings ok=100000, missing_or_wrong=0, total_nonempty=100000


image_features_part_2: embeddings ok=51500, missing_or_wrong=0, total_nonempty=51500


image_features_part_3: embeddings ok=50000, missing_or_wrong=0, total_nonempty=50000


image_features_part_4: embeddings ok=50000, missing_or_wrong=0, total_nonempty=50000


image_features_part_5: embeddings ok=50004, missing_or_wrong=0, total_nonempty=50004


image_features_part_6: embeddings ok=50000, missing_or_wrong=0, total_nonempty=50000


image_features_part_7: embeddings ok=8250, missing_or_wrong=0, total_nonempty=8250


In [ ]:

"""
Geht über alle image_features_part_% Tabellen,
zählt pro Spalte (alle!) filled/missing
und exportiert fehlende Pfade je Spalte in eine CSV.
Erzeugt:
- all_columns_summary.csv  (long format: table, column, col_type, total, filled, missing)
- missing_by_column_paths.csv (table, column, id, filename, path)
"""

import os, csv, sqlite3
from tqdm import tqdm

# ====== CONFIG ======
DB_PATH = r"C:\BIG_DATA\data\database.db"
TABLE_PREFIX = "image_features_part_"
OUTPUT_DIR = "hash_reports_all_columns"
USE_TQDM = True

os.makedirs(OUTPUT_DIR, exist_ok=True)
SUMMARY_CSV = os.path.join(OUTPUT_DIR, "all_columns_summary.csv")
MISSING_CSV = os.path.join(OUTPUT_DIR, "missing_by_column_paths.csv")

# ====== DB helpers ======
def get_shard_tables(cur):
    cur.execute(
        "SELECT name FROM sqlite_master WHERE type='table' AND name LIKE ? ORDER BY name ASC",
        (f"{TABLE_PREFIX}%",),
    )
    return [r[0] for r in cur.fetchall()]

def get_columns(cur, table):
    """
    Liefert Liste von (name, decl_type) aus PRAGMA table_info.
    decl_type kann z.B. 'TEXT', 'INTEGER', 'REAL', 'NUMERIC', 'BLOB' etc. sein.
    """
    cur.execute(f"PRAGMA table_info({table});")
    cols = [(row[1], (row[2] or "").upper()) for row in cur.fetchall()]
    # 'id' rauswerfen
    return [(n, t) for (n, t) in cols if n.lower() != "id"]

def count_total(cur, table):
    cur.execute(f"SELECT COUNT(*) FROM {table}")
    return int(cur.fetchone()[0])

def build_missing_condition(col_name, col_type_upper):
    # TEXT -> Null oder leere/Whitespace-Strings; sonst nur NULL
    if "TEXT" in col_type_upper:
        return f"({col_name} IS NULL OR TRIM({col_name}) = '')"
    else:
        return f"({col_name} IS NULL)"

def column_exists(cur, table, col_name):
    cur.execute(f"PRAGMA table_info({table});")
    return any(r[1] == col_name for r in cur.fetchall())

# ====== Export missing paths (streaming) ======
def export_missing_for_column(cur, table, col_name, cond, writer, has_filename, has_path):
    # Baue SELECT dynamisch je nach vorhandenen Spalten
    select_cols = ["id"]
    if has_filename: select_cols.append("filename")
    if has_path:     select_cols.append("path")
    sel = ", ".join(select_cols)

    cur.execute(f"SELECT {sel} FROM {table} WHERE {cond}")
    while True:
        rows = cur.fetchmany(5000)
        if not rows:
            break
        for row in rows:
            # row ist Tupel mit 1..3 Elementen je nach vorhandenen Spalten
            rid = row[0]
            fname = row[1] if has_filename else ""
            pth = row[2] if (has_filename and has_path and len(row) > 2) else (row[1] if (has_path and not has_filename and len(row) > 1) else "")
            writer.writerow({"table": table, "column": col_name, "id": rid, "filename": fname, "path": pth})

def main():
    conn = sqlite3.connect(DB_PATH, isolation_level=None)
    cur = conn.cursor()

    tables = get_shard_tables(cur)
    if not tables:
        print("Keine Shard-Tabellen gefunden.")
        return

    # Prepare CSVs
    with open(SUMMARY_CSV, "w", newline="", encoding="utf-8") as fsum, \
         open(MISSING_CSV, "w", newline="", encoding="utf-8") as fmiss:

        sum_writer = csv.DictWriter(fsum, fieldnames=["table", "column", "col_type", "total", "filled", "missing"])
        sum_writer.writeheader()

        miss_writer = csv.DictWriter(fmiss, fieldnames=["table", "column", "id", "filename", "path"])
        miss_writer.writeheader()

        iterator = tqdm(tables, desc="Tabellen", unit="tbl") if USE_TQDM else tables
        for tbl in iterator:
            # Spalten & Basisinfos
            cols = get_columns(cur, tbl)
            total = count_total(cur, tbl)

            # Prüfen, ob filename/path vorhanden sind (für Export)
            present_colnames = {name for (name, _) in cols}
            has_filename = "filename" in present_colnames
            has_path = "path" in present_colnames

            # Fortschritt pro Tabelle (über Spalten)
            col_iter = tqdm(cols, desc=f"{tbl}", unit="col", leave=False) if USE_TQDM else cols
            for col_name, col_type in col_iter:
                cond = build_missing_condition(col_name, col_type)

                # Missing zählen
                cur.execute(f"SELECT COUNT(*) FROM {tbl} WHERE {cond}")
                missing = int(cur.fetchone()[0])
                filled = total - missing

                sum_writer.writerow({
                    "table": tbl,
                    "column": col_name,
                    "col_type": col_type,
                    "total": total,
                    "filled": filled,
                    "missing": missing
                })

                # Fehlende Pfade exportieren
                if missing > 0:
                    export_missing_for_column(cur, tbl, col_name, cond, miss_writer, has_filename, has_path)

    conn.close()
    
    print(f"[OK] Summary: {SUMMARY_CSV}")
    print(f"[OK] Missing paths (alle Spalten): {MISSING_CSV}")

if __name__ == "__main__":
    main()


Tabellen: 100%|██████████| 7/7 [00:13<00:00,  1.94s/tbl]

[OK] Summary: hash_reports_all_columns\all_columns_summary.csv
[OK] Missing paths (alle Spalten): hash_reports_all_columns\missing_by_column_paths.csv


In [2]:
import os
import sqlite3
from pathlib import Path
from tqdm import tqdm
"""
Repariert embedding_path in allen image_features_part_% Tabellen – ID-weise paged,
damit Updates den Scan nicht abbrechen. Loggt fehlende als Bildpfade.
"""
# ===== CONFIG =====
DB_PATH = r"C:\BIG_DATA\data\database.db"
TABLE_PREFIX = "image_features_part_"
EMBEDDING_DIR = Path(r"C:\BIG_DATA\embeddings")
MISSING_TXT = "missing_embedding_images.txt"
PAGE = 5000  # Seitengröße nach id

# ===== Helpers =====
def get_tables(cur):
    cur.execute("SELECT name FROM sqlite_master WHERE type='table' AND name LIKE ? ORDER BY name ASC",
                (f"{TABLE_PREFIX}%",))
    return [r[0] for r in cur.fetchall()]

def col_exists(cur, table, col):
    cur.execute(f"PRAGMA table_info({table});")
    return any(r[1] == col for r in cur.fetchall())

def index_embeddings(root: Path):
    idx = {}
    for base, _, files in os.walk(root):
        for fn in files:
            if fn.lower().endswith(".npy"):
                full = os.path.join(base, fn)
                idx.setdefault(fn, full)  # erster Treffer gewinnt
    return idx

def expected_basenames(filename: str):
    fn = (filename or "").strip()
    stem = Path(fn).stem
    return [f"{fn}.npy", f"{stem}.npy"]

def is_ok(embedding_path: str, filename: str) -> bool:
    if not embedding_path:
        return False
    p = embedding_path.strip()
    if not p or not p.lower().endswith(".npy"):
        return False
    if not os.path.exists(p):
        return False
    base = os.path.basename(p)
    return base in set(expected_basenames(filename))

def try_fix_path(p_current: str, filename: str, idx: dict, embedding_dir: Path):
    fn = (filename or "").strip()
    stem = Path(fn).stem
    cand_names = [f"{fn}.npy", f"{stem}.npy"]

    # 1) alter Pfad + .npy?
    if p_current:
        p_strip = p_current.strip()
        if p_strip and not p_strip.lower().endswith(".npy"):
            cand = p_strip + ".npy"
            if os.path.exists(cand):
                return cand

    # 2) direkt im Zielordner
    for name in cand_names:
        p = embedding_dir / name
        if os.path.exists(p):
            return str(p)

    # 3) Index
    for name in cand_names:
        if name in idx:
            return idx[name]

    # 4) eindeutiger Prefix-Treffer (stem_)
    stem_prefix = f"{stem}_"
    matches = [path for base, path in idx.items() if base.startswith(stem_prefix)]
    if len(matches) == 1:
        return matches[0]

    return None

# ===== Main per Tabelle (paged) =====
def process_table(conn, tbl, idx, missing_file):
    cur = conn.cursor()
    cur_write = conn.cursor()

    # Zählung für Progressbar
    cur.execute(f"SELECT COUNT(*) FROM {tbl}")
    total = int(cur.fetchone()[0])
    pbar = tqdm(total=total, desc=tbl, unit="row")

    ok = fixed = missing = 0
    last_id = 0

    while True:
        # Page nach Primärschlüssel
        cur.execute(
            f"""SELECT id, filename, path, embedding_path
                FROM {tbl}
                WHERE id > ?
                ORDER BY id ASC
                LIMIT ?""",
            (last_id, PAGE)
        )
        rows = cur.fetchall()
        if not rows:
            break

        updates = []  # (new_path, id)
        for _id, filename, img_path, emb_path in rows:
            if is_ok(emb_path, filename):
                ok += 1
                pbar.update(1)
                last_id = _id
                continue

            newp = try_fix_path(emb_path or "", filename or "", idx, EMBEDDING_DIR)
            if newp:
                updates.append((newp, _id))
                fixed += 1
            else:
                # fehlend -> Bildpfad loggen
                missing += 1
                missing_file.write(f"{_id}; {tbl}; {filename or ''}; {img_path or ''}\n")

            pbar.update(1)
            last_id = _id

        if updates:
            cur_write.executemany(f"UPDATE {tbl} SET embedding_path=? WHERE id=?", updates)
            conn.commit()

    pbar.close()
    print(f"{tbl}: ok={ok:,}, fixed={fixed:,}, missing={missing:,}, total={total:,}")
    return ok, fixed, missing

def main():
    print("Indexiere Embeddings im Ordner …")
    idx = index_embeddings(EMBEDDING_DIR)
    print(f"Gefundene Embeddings: {len(idx):,}")

    conn = sqlite3.connect(DB_PATH, isolation_level=None)
    cur = conn.cursor()
    tables = get_tables(cur)
    if not tables:
        print("Keine Tabellen gefunden.")
        return

    # Spalten-Check
    for t in tables:
        for c in ("filename", "path", "embedding_path"):
            if not col_exists(cur, t, c):
                print(f"[WARN] {t}: Spalte {c} fehlt – übersprungen.")
                tables.remove(t)
                break

    grand_ok = grand_fixed = grand_missing = 0
    with open(MISSING_TXT, "w", encoding="utf-8") as mf:
        mf.write("# id; table; filename; image_path\n")
        for tbl in tables:
            ok, fixed, missing = process_table(conn, tbl, idx, mf)
            grand_ok += ok
            grand_fixed += fixed
            grand_missing += missing

    conn.close()

    print("\n=== Zusammenfassung ===")
    print(f"OK:       {grand_ok:,}")
    print(f"Fixed:    {grand_fixed:,}")
    print(f"Missing:  {grand_missing:,}")
    print(f"Fehlende Bildpfade in: {MISSING_TXT}")

if __name__ == "__main__":
    main()

Indexiere Embeddings im Ordner …
Gefundene Embeddings: 359,474


image_features_part_1: 100%|██████████| 100000/100000 [00:07<00:00, 13850.67row/s]


image_features_part_1: ok=100,000, fixed=0, missing=0, total=100,000


image_features_part_2: 100%|██████████| 51500/51500 [00:03<00:00, 14453.34row/s]


image_features_part_2: ok=51,500, fixed=0, missing=0, total=51,500


...

image_features_part_5: ok=50,004, fixed=0, missing=0, total=50,004


image_features_part_6: 100%|██████████| 50000/50000 [02:34<00:00, 324.63row/s]  


image_features_part_6: ok=5,000, fixed=45,000, missing=0, total=50,000


image_features_part_7: 100%|██████████| 8250/8250 [00:11<00:00, 718.09row/s]  

image_features_part_7: ok=5,000, fixed=3,250, missing=0, total=8,250

=== Zusammenfassung ===
OK:       311,504
Fixed:    48,250
Missing:  0
Fehlende Bildpfade in: missing_embedding_images.txt
